<a href="https://colab.research.google.com/github/alexeiveselov92/Yandex-Data-Analysis/blob/master/%D0%92%D0%BD%D0%B5%D0%B4%D1%80%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%87%D0%B0%D1%82%D0%B0%20%D1%81%20%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80%D0%BE%D0%BC%20%D0%BD%D0%B0%20%D1%81%D0%B0%D0%B9%D1%82%20-%20AB%20%D1%82%D0%B5%D1%81%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%80%D0%B5%D0%B7%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A/B тестирование внедрения чата с менеджером на сайт**

<a id='content'></a>
## Оглавление
* [Описание целей исследования](#description)
* [Загрузка библиотек](#download)
* [Спецификация](#specification)
* [Предобработка](#preprocessing)
* [Проведем исследовательский анализ данных](#research)
  * Исследуем конверсию в воронке на разных этапах
  * Обладают ли выборки одинаковыми распределениями количества событий на пользователя?
  * Как число событий распределено по дням?
  * Подумаем, есть ли какие-то нюансы данных, которые нужно учесть, прежде чем приступать к A/B-тестированию?
* [Проведем оценку результатов A/B-тестирования](#test)
  * Что можно сказать про результаты A/A-тестирования?
  * Проверим статистическую разницу долей z-критерием
* [Опишем выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования](#results)

<a id='description'></a>
## Описание целей исследования
[К оглавлению](#content)

<a id='download'></a>
## Загрузка библиотек
[К оглавлению](#content)



In [0]:
import pandas as pd
import math as mth
from scipy import stats as st
import random
import numpy as np

#визуализация
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px

#другое
pd.options.display.expand_frame_repr = False
import warnings
warnings.filterwarnings('ignore')

#загрузка файлов с диска
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<a id='specification'></a>
## Спецификация
[К оглавлению](#content)
* **`Event`** - тип события
  * `StartReadingOffer`
  * `OpenProductCard`
  * `OpenPaymentScreen`
  * `StartCheckProduct`
* **`user_id`** - id пользователя
* **`EventTime`** - время события
* **`Group`** - группа события
  * `A1` - контрольная группа
  * `A2` - контрольная группа
  * `B` - экспериментальная группа

<a id='preprocessing'></a>
## Предобработка данных
[К оглавлению](#content)

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/final/ab_log.csv')
df.columns = df.columns.str.lower()
df['eventtime'] = pd.to_datetime(df['eventtime'],unit='s')
df['date'] = pd.to_datetime(df['eventtime'].dt.date)
df.info()
print('\nДубликатов в выгрузке: {}'.format(df.duplicated().sum()))
df = df.drop_duplicates().reset_index(drop = True)
print('Дубликаты удалены, дубликатов осталось: {}'.format(df.duplicated().sum()))
print('Пропущенных значений не обнаружено')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154788 entries, 0 to 154787
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   event      154788 non-null  object        
 1   user_id    154788 non-null  int64         
 2   eventtime  154788 non-null  datetime64[ns]
 3   group      154788 non-null  object        
 4   date       154788 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 5.9+ MB

Дубликатов в выгрузке: 296
Дубликаты удалены, дубликатов осталось: 0
Пропущенных значений не обнаружено


<a id='research'></a>
## Исследовательский анализ данных
[К оглавлению](#content)

* Исследуем конверсию в воронке на разных этапах
* Обладают ли выборки одинаковыми распределениями количества событий на пользователя?
* Как число событий распределено по дням?
* Подумаем, есть ли какие-то нюансы данных, которые нужно учесть, прежде чем приступать к A/B-тестированию?


In [0]:
print('Всего событий в логе: {}'.format(len(df)))
print('Всего пользователей: {}'.format(len(df.user_id.unique())))
print('В среднем событий на одного пользователя: {}'.format(df.groupby('user_id')['event'].count().mode()[0]))
for group in df['group'].unique():
  print('Кол-во событий в группе {}: {}'.format(group , len(df[df['group']==group])))
print('Минимальная дата: {}'.format(df['date'].min()))
print('Максимальная дата: {}'.format(df['date'].max()))
print('Период составляет: {}'.format(df['eventtime'].max()-df['eventtime'].min()))
event_date_grouped = df.groupby('date', as_index=False).agg({'eventtime':'count'})
event_date_grouped['date_text'] = event_date_grouped['date'].dt.strftime('%d.%m')
fig = px.bar(event_date_grouped, x = 'date', y='eventtime', text='date_text', width=750, height=450)
fig.update_layout(
    title={
        'text': "Распределение количества событий по дням",
        'xanchor': 'center',
        'x':0.5
    },
    yaxis_title="Количество событий",
    xaxis_title="Дата")
fig.update_traces(textposition='outside')
fig.show()

Всего событий в логе: 154492
Всего пользователей: 6805
В среднем событий на одного пользователя: 6
Кол-во событий в группе A2: 49828
Кол-во событий в группе B: 55018
Кол-во событий в группе A1: 49646
Минимальная дата: 2019-07-25 00:00:00
Максимальная дата: 2019-08-05 00:00:00
Период составляет: 11 days 06:46:54


> Видим, что данные предоставллены полноценно только за 5 дней с 01.08 по 05.08, остальные дни отбросим

In [0]:
filtered_df = df[df['date']>='2019-08-01']
funnel = filtered_df.groupby('event').agg({'user_id':'nunique', 'eventtime':'count'}).sort_values(by='user_id', ascending=False)
funnel = funnel.loc[['StartReadingOffer', 'StartCheckProduct', 'OpenProductCard', 'OpenPaymentScreen']]
funnel.columns = ['users', 'events']
funnel['change_users'] = funnel['users'].pct_change() + 1
funnel['change_events'] = funnel['events'].pct_change() + 1
funnel = funnel.fillna(1)
funnel = funnel.reset_index()
print(funnel)
fig = go.Figure()
fig.add_trace(go.Funnel(
    y = funnel['event'],
    x = funnel['events'],
    textinfo = "value+percent previous"))
fig.update_layout(title={'text': "Воронка событий", 'xanchor': 'center', 'x':0.5, 'y':0.87})
fig.show()
fig = go.Figure()
fig.add_trace(go.Funnel(
    y = funnel['event'],
    x = funnel['users'],
    textinfo = "value+percent previous"))
fig.update_layout(title={'text': "Воронка пользователей", 'xanchor': 'center', 'x':0.5, 'y':0.87})
fig.show()

               event  users  events  change_users  change_events
0  StartReadingOffer   6592   72281      1.000000       1.000000
1  StartCheckProduct   3972   29641      0.602549       0.410080
2    OpenProductCard   3229   27334      0.812941       0.922169
3  OpenPaymentScreen   3340   22422      1.034376       0.820297


> 1.   Видим странную особенность - экран оплаты увидело бОльшее кол-во пользователей, чем карточку самого продукта - возможно, у пользователей есть возможность совершить оплату раньше просмотра самого продукта и такая разница возникает из-за повторных покупок пользователей, которые оплачивают сразу не переходя в карточку продукта.
2.   Больше всего пользователей теряется на перввом этапе - StartReadingOffer - StartCheckProduct - около 40\%



In [0]:
events_group_by_users = df.pivot_table(index='user_id', columns='event', values = 'eventtime', aggfunc='count')
events_by_buyers = events_group_by_users[events_group_by_users['OpenPaymentScreen']>0]
def whats_more(row):
  if row['OpenPaymentScreen'] > row['OpenProductCard']:
    return True
  else:
    return False

events_by_buyers['whats_more'] = events_by_buyers.apply(whats_more, axis=1)
print('Покупателей, которые экран покупки смотрели чаще экрана продукта - {:.2%}'.format(events_by_buyers['whats_more'].mean()))
print('Покупатели всего посмотрели экран оплаты - {}, в среднем просмотров на покупателя - {}\nПокупатели всего посмотрели экран продукта - {}, в среднем просмотров на покупателя - {}'.format(
    events_by_buyers['OpenPaymentScreen'].sum(), events_by_buyers['OpenPaymentScreen'].median(),
    events_by_buyers['OpenProductCard'].sum(), events_by_buyers['OpenProductCard'].median()
))


Покупателей, которые экран покупки смотрели чаще экрана продукта - 3.12%
Покупатели всего посмотрели экран оплаты - 22635.0, в среднем просмотров на покупателя - 3.0
Покупатели всего посмотрели экран продукта - 27454.0, в среднем просмотров на покупателя - 4.0


<a id='test'></a>
## Проведем оценку результатов A/B-тестирования
[К оглавлению](#content)
* Что можно сказать про результаты A/A-тестирования?
* Проверим статистическую разницу долей z-критерием


In [0]:
event_parts_per_groups = filtered_df.pivot_table(index='event', values='user_id', columns='group', aggfunc='nunique')
event_parts_per_groups = event_parts_per_groups.loc[['StartReadingOffer', 'StartCheckProduct', 'OpenProductCard', 'OpenPaymentScreen']]
event_parts_per_groups = event_parts_per_groups.reset_index()

users_by_group_A1 = len(filtered_df[filtered_df['group']=='A1']['user_id'].unique())
users_by_group_A2 = len(filtered_df[filtered_df['group']=='A2']['user_id'].unique())
users_by_group_B = len(filtered_df[filtered_df['group']=='B']['user_id'].unique())
event_parts_per_groups['A1_A2'] = event_parts_per_groups['A1'] + event_parts_per_groups['A2']
users_by_group_A1_A2 = users_by_group_A1 + users_by_group_A2
event_parts_per_groups['A1_part'] = event_parts_per_groups['A1'] / users_by_group_A1
event_parts_per_groups['A2_part'] = event_parts_per_groups['A2'] / users_by_group_A2
event_parts_per_groups['B_part'] = event_parts_per_groups['B'] / users_by_group_B
event_parts_per_groups = event_parts_per_groups.set_index('event')
event_parts_per_groups['A1_A2_part'] = event_parts_per_groups['A1_A2'] / users_by_group_A1_A2
print(event_parts_per_groups)

print('\nПроверим нулевые гипотезы о том, что доли событий в группах A1 и A2 равны по-очередно:\n')
alpha =0.05

def Z_test(index, group_1, group_2, part_1, part_2, number_1, number_2):
    print('{}:'.format(index))
    p1 = event_parts_per_groups[part_1][index]
    p2 = event_parts_per_groups[part_2][index]
    p = (event_parts_per_groups[group_1][index]+event_parts_per_groups[group_2][index])/\
    (number_1 + number_2)
    n1 = number_1
    n2 = number_2
    z_value = (p1-p2)/mth.sqrt(p * (1-p) * (1/n1 + 1/n2))
    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('p-значение: ', p_value)
    if p_value < alpha:
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

Z_test('StartReadingOffer', 'A1', 'A2', 'A1_part', 'A2_part', users_by_group_A1, users_by_group_A2)
Z_test('StartCheckProduct', 'A1', 'A2', 'A1_part', 'A2_part', users_by_group_A1, users_by_group_A2)
Z_test('OpenProductCard', 'A1', 'A2', 'A1_part', 'A2_part', users_by_group_A1, users_by_group_A2)
Z_test('OpenPaymentScreen', 'A1', 'A2', 'A1_part', 'A2_part', users_by_group_A1, users_by_group_A2)



group                A1    A2     B  A1_A2   A1_part   A2_part    B_part  A1_A2_part
event                                                                               
StartReadingOffer  2206  2166  2220   4372  0.978271  0.977437  0.980132    0.977857
StartCheckProduct  1318  1339  1315   2657  0.584479  0.604242  0.580574    0.594274
OpenProductCard    1071  1085  1073   2156  0.474945  0.489621  0.473731    0.482219
OpenPaymentScreen  1112  1132  1096   2244  0.493126  0.510830  0.483885    0.501901

Проверим нулевые гипотезы о том, что доли событий в группах A1 и A2 равны по-очередно:

StartReadingOffer:
p-значение:  0.8497705060264933
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
StartCheckProduct:
p-значение:  0.17844987219688857
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
OpenProductCard:
p-значение:  0.3261357129982716
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
OpenPaymentScre

> * Разницы между группами A1 и A2 не обнаружено
* Хорошо, АА тест пройден, можно переходить к АВ тесту

In [0]:
print('Сравним группу B с контрольной группой A1')

print('\nПроверим нулевые гипотезы о том, что доли событий в группах A1 и B равны по-очередно:')
alpha =0.05
Z_test('StartReadingOffer', 'A1', 'B', 'A1_part', 'B_part', users_by_group_A1, users_by_group_B)
Z_test('StartCheckProduct', 'A1', 'B', 'A1_part', 'B_part', users_by_group_A1, users_by_group_B)
Z_test('OpenProductCard', 'A1', 'B', 'A1_part', 'B_part', users_by_group_A1, users_by_group_B)
Z_test('OpenPaymentScreen', 'A1', 'B', 'A1_part', 'B_part', users_by_group_A1, users_by_group_B)
print('\nРазницы между группами A1 и B не обнаружено\n\n')

print('Сравним группу B с контрольной группой A2')

print('\nПроверим нулевые гипотезы о том, что доли событий в группах A2 и B равны по-очередно:')
alpha =0.05
Z_test('StartReadingOffer', 'A2', 'B', 'A2_part', 'B_part', users_by_group_A2, users_by_group_B)
Z_test('StartCheckProduct', 'A2', 'B', 'A2_part', 'B_part', users_by_group_A2, users_by_group_B)
Z_test('OpenProductCard', 'A2', 'B', 'A2_part', 'B_part', users_by_group_A2, users_by_group_B)
Z_test('OpenPaymentScreen', 'A2', 'B', 'A2_part', 'B_part', users_by_group_A2, users_by_group_B)
print('\nРазницы между группами A2 и B не обнаружено\n\n')

print('Сравним группу B с объединенной контрольной группой A1_A2')

print('\nПроверим нулевые гипотезы о том, что доли событий в группах A1_A2 и B равны по-очередно:')
alpha =0.05
Z_test('StartReadingOffer', 'A1_A2', 'B', 'A1_A2_part', 'B_part', users_by_group_A1_A2, users_by_group_B)
Z_test('StartCheckProduct', 'A1_A2', 'B', 'A1_A2_part', 'B_part', users_by_group_A1_A2, users_by_group_B)
Z_test('OpenProductCard', 'A1_A2', 'B', 'A1_A2_part', 'B_part', users_by_group_A1_A2, users_by_group_B)
Z_test('OpenPaymentScreen', 'A1_A2', 'B', 'A1_A2_part', 'B_part', users_by_group_A1_A2, users_by_group_B)
print('\nРазницы между объединенной группой A1_A2 и B не обнаружено\n\n')

Сравним группу B с контрольной группой A1

Проверим нулевые гипотезы о том, что доли событий в группах A1 и B равны по-очередно:
StartReadingOffer:
p-значение:  0.6609483968688323
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
StartCheckProduct:
p-значение:  0.7900962987501854
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
OpenProductCard:
p-значение:  0.934870935427434
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
OpenPaymentScreen:
p-значение:  0.5342996314330497
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Разницы между группами A1 и B не обнаружено


Сравним группу B с контрольной группой A2

Проверим нулевые гипотезы о том, что доли событий в группах A2 и B равны по-очередно:
StartReadingOffer:
p-значение:  0.5311297624671363
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
StartCheckProduct:
p-значение:  0.10697657408290495
Н

> * В рамках множественного сравнения групп мы использовали критический уровень значимости 0.05.
* При множественном тестировании иногда используют поправки критического уровня значимости (уменьшение его)
для снижения вероятности ошибки первого рода. 
* В нашем случае p-value в каждом тесте достаточно высок,
для того, чтобы введение поправок критического уровня значимости было бессмысленным. Это не изменит результаты.


<a id='results'></a>
## Опишем выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования
[К оглавлению](#content)

На этапе подготовки данных мы выявили несущественное количество дублей строк и удалили их.
Всего событий в логе: 154492. Всего пользователей: 6805. В среднем событий на одного пользователя: 6.
* Кол-во событий в группе A2: 49828
* Кол-во событий в группе B: 55018
* Кол-во событий в группе A1: 49646

Данные в логе были за период в 2 недели, но мы выяснили, что полноценные данные были лишь за 5 дней, поэтому неполноценные данные мы отсекли и взяли период с 1 по 5 августа. Количество данных уменьшилось несущественно.

Больше всего пользователей теряем на первом шаге воронки  StartReadingOffer - StartCheckProduct - около 40%. До оплаты же доходит около 50% пользователей.

Критический уровень значимости перед проведением тестирований мы взяли 0.05. Перед A/B тестированием мы провели A/A тестирование групп  и выяснилось, что всё впорядке и распределение на группы было корректным.

**Мы провели A/B тестирование и не заметили статистически значимых отличий между контрольными группами и экспериментальной. Это значит, что внедрение чата с менеджером на сайт не дало никакого эффекта.**

Проводя множественное сравнение при A/A и A/B тестированиях, мы не использовали никакие поправки критического уровня значимости, так как в этом не было никакого смысла - при уменьшении критического уровня значимости с помощью имеющихся поправок результаты тестирования не изменились бы, это видно из полученных p-value (минимальное p-value - 0.10).
